In [1]:
import numpy as np
import xarray as xr
import time 
import os
from netCDF4 import Dataset
from pyproj import CRS
import rasterio
import rioxarray
from rasterio.enums import Resampling
from sklearn.preprocessing import StandardScaler

In [2]:
#For calculating statistical northness and eastness

filename_physstates_2d = '/cluster/projects/itk-SINMOD/coral-mapping/midnor/samp_2D_jan_jun.nc'
physstates_2d = Dataset(filename_physstates_2d, 'r')
gridLons = physstates_2d.variables['gridLons']

In [3]:
filename_sinmod = '/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc'
sinmid = xr.open_dataset(filename_sinmod)

In [4]:
print(list(sinmid.variables))

['time', 'grid_mapping', 'LayerDepths', 'xc', 'yc', 'zc', 'depth', 'DXxDYy', 'u_velocity', 'v_velocity', 'elevation', 'temperature', 'salinity', 'ice_thickness', 'ice_compactness', 'salinity_ice']


In [8]:
#Create bottom-features 

def process_bottom_layer(filepath,
    variable_name,
    gridLons=None,  # To calculate statistical northness and eastness
    output_path=None
):
    """
    Process bottom layer data for a specified variable in a NetCDF file.

    Parameters:
    - filepath (str): Path to the NetCDF file.
    - variable_name (str): Name of the variable to process.
    - output_path (str): Path to save the processed file (optional). If None, the result is not saved.
    
    Returns:
    - xarray.DataArray: The time-averaged bottom layer data.
    """

    # Check if output path is valid
    if output_path is not None:
        output_dir = os.path.dirname(output_path)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        elif not os.access(output_dir, os.W_OK):
            raise PermissionError(f"Write permission denied for the directory: {output_dir}")

    time_start = time.time()

    ds = xr.open_dataset(filepath)
    print(f"\nAccessed the dataset after {time.time() - time_start:.2f} seconds")

    # Drop NaN values along specified dimensions
    #for dim in ["time", "xc", "yc"]:
     #   if dim in ds.dims:  
      #      ds = ds.dropna(dim=dim, how="any")

    # Extract the desired variable
    if variable_name == "current_speed" or variable_name in ["statistical_northness", "statistical_eastness"]:
        data_var = ds["u_velocity"]
    elif variable_name == "temperature_sundahl":
        data_var = ds["temperature"]
    else:
        data_var = ds[variable_name]

    # Extract the first time step
    time_slice = data_var.isel(time=0)

    # Create a mask for valid values in the first time step
    valid_mask = ~time_slice.isnull()

    # Find the index of the bottom-most valid layer for each (yc, xc)
    bottom_layer_idx = valid_mask.argmin(dim="zc") - 1

    # Ensure bottom_layer_idx does not go negative
    bottom_layer_idx = bottom_layer_idx.clip(min=0)

    # Extract the bottom layer data across all time steps
    if variable_name == "current_speed":
        bottom_layer_data = (data_var.isel(zc=bottom_layer_idx) ** 2 + ds["v_velocity"].isel(zc=bottom_layer_idx) ** 2) ** 0.5
    elif variable_name in ["statistical_northness", "statistical_eastness"]:
        longitude_of_projection_origin = ds["grid_mapping"].attrs["longitude_of_projection_origin"]
        theta = gridLons - longitude_of_projection_origin
        eastward_velocity = data_var.isel(zc=bottom_layer_idx) * np.cos(np.deg2rad(theta)) - ds["v_velocity"].isel(zc=bottom_layer_idx) * np.sin(np.deg2rad(theta))
        northward_velocity = data_var.isel(zc=bottom_layer_idx) * np.sin(np.deg2rad(theta)) + ds["v_velocity"].isel(zc=bottom_layer_idx) * np.cos(np.deg2rad(theta))
        aspect = np.arctan2(eastward_velocity, northward_velocity)

        if variable_name == 'statistical_eastness':
            bottom_layer_data = np.sin(aspect)
        else:
            bottom_layer_data = np.cos(aspect)
    else:
        bottom_layer_data = data_var.isel(zc=bottom_layer_idx)

    ds.close()

    print(f"\nExtracted the bottom layer data after {time.time() - time_start:.2f} seconds.\n\nStarting computation of statistics...")

    if variable_name == 'temperature_sundahl':
        print("variable is sundahl temp")

        march_may_data = bottom_layer_data.isel(time=slice(59, 151))
        oct_dec_data = bottom_layer_data.isel(time=slice(273, 365))
        print("extracted months")
        """
        # Convert time dimension to datetime64 if it's not already
        if not np.issubdtype(bottom_layer_data['time'].dtype, np.datetime64):
            bottom_layer_data['time'] = xr.decode_cf(bottom_layer_data)['time']
        
        # Extract March-May and October-December using time accessor
        march_may_data = bottom_layer_data.where(bottom_layer_data['time'].dt.month.isin([3, 4, 5]), drop=True)
        oct_dec_data = bottom_layer_data.where(bottom_layer_data['time'].dt.month.isin([10, 11, 12]), drop=True)
        """
        # Compute seasonal means
        mean_march_may = march_may_data.mean(dim="time", skipna=True)
        mean_oct_dec = oct_dec_data.mean(dim="time", skipna=True)
        print("means extracted")
    
        print(f"\nComputed statistics after {time.time() - time_start:.2f} seconds")
    
        # Concatenate with explicit naming
        stats_array = xr.concat([mean_march_may, mean_oct_dec], dim="stat").rename(f"{variable_name}_features")
        stats_array = stats_array.assign_coords(stat=["min_(mean_march_may)", "max_(mean_oct_dec)"])
    else:
        # Calculate statistics across time
        time_avg_bottom_layer = bottom_layer_data.mean(dim="time", skipna=True)

        # Calculate both 10th and 90th percentiles
        time_percentiles = bottom_layer_data.quantile([0.1, 0.9], dim="time", skipna=True)

        print(f"\nComputed statistics after {time.time() - time_start:.2f} seconds")

        # Concatenate mean and percentiles into a DataArray
        stats_array = xr.concat(
            [time_avg_bottom_layer, time_percentiles.sel(quantile=0.1).drop_vars("quantile"), 
             time_percentiles.sel(quantile=0.9).drop_vars("quantile")],
            dim="stat"
        ).rename(f"{variable_name}_features")
        stats_array = stats_array.assign_coords(stat=["mean", "10th_percentile", "90th_percentile"])

    # Save to output file if specified
    if output_path:
        stats_array.to_netcdf(output_path, mode='w')

    return stats_array

In [20]:
#Run for desired features

#Run on temperature data (sundahl definition)
#process_bottom_layer("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc", "temperature_sundahl", output_path="/cluster/home/maikents/features_midnor_2019/bottom_features/temperature_sundahl_bottom_features.nc")

#Run on statistical northness
#process_bottom_layer("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc", "statistical_northness", gridLons, output_path="/cluster/home/maikents/features_midnor_2019/bottom_features/statistical_northness_bottom_features.nc")

#Run on statistical eastness
#process_bottom_layer("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc", "statistical_eastness", gridLons, output_path="/cluster/home/maikents/features_midnor_2019/bottom_features/statistical_eastness_bottom_features.nc")

#Run on salinity
#process_bottom_layer("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc", "salinity", gridLons, output_path="/cluster/home/maikents/features_midnor_2019/bottom_features/salinity_bottom_features.nc")

#Run on current speed
process_bottom_layer("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc", "current_speed", output_path="/cluster/home/maikents/features_midnor_2019/bottom_features/current_speed_bottom_features.nc")



Accessed the dataset after 0.01 seconds

Extracted the bottom layer data after 413.52 seconds.

Starting computation of statistics...


/cluster/home/maikents/PyEnvCoralMapping/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1620: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,



Computed statistics after 435.79 seconds


<xarray.DataArray 'current_speed_features' (stat: 3, yc: 555, xc: 950)> Size: 6MB
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [0.06207896, 0.06205925, 0.06194371, ..., 0.04004636,
         0.04026011, 0.04036504],
        [0.06207896, 0.06213063, 0.06211492, ..., 0.04028086,
         0.04029776, 0.04049896],
        [0.06207896, 0.06226798, 0.06227183, ..., 0.04058979,
         0.04058235, 0.04090336]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [0.02093785, 0.02207078, 0.0220333 , ..., 0.01346248,
         0.01405384, 0.01339338],
        [0.02093785, 0.02218143, 0.0220049 , ..., 0.01345605,
         0.01368445, 0.01327797],
        [0.02093785, 0.02177577, 0.02183845, ..., 0.01372578,
         0.01369959, 0.01371343]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [0.11262337, 0.11233231, 0.11176776, ..., 0.0690499 ,
         0.06966082, 0.06935164],
        [0.11262337, 0.11213985, 0.11178505, ..., 0.06996491,
         0.06935545, 0.06958441],
        [0.11262337, 0.11201465, 0.11206156, ..., 0.07046502,
         0.07041683, 0.07024004]]], shape=(3, 555, 950), dtype=float32)
Coordinates:
  * xc       (xc) float32 4kB 800.0 1.6e+03 2.4e+03 ... 7.592e+05 7.6e+05
  * yc       (yc) float32 2kB 800.0 1.6e+03 2.4e+03 ... 4.432e+05 4.44e+05
    zc       (yc, xc) float32 2MB 3.0 3.0 3.0 3.0 ... 2e+03 2e+03 2e+03 2e+03
  * stat     (stat) <U15 180B 'mean' '10th_percentile' '90th_percentile'

In [10]:
temp = xr.open_dataset("/cluster/home/maikents/features_midnor_2019/bottom_features/temperature_sundahl_bottom_features.nc")

In [ ]:
#Align SINMOD and EMOD data into same format. Returns a big array of all the features including EMOD and SINMOD, 
#with the same grid spacing and EPRSG.

In [ ]:
#1: Load and combine all bottom features 

#Load the SINMOD NetCDF file
sinmod_file = '/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc'
temp_file = 'processed_data/features/temperature_bottom_features.nc'
salinity_file = 'processed_data/features/salinity_bottom_features.nc'
current_speed_file = 'processed_data/features/current_speed_bottom_features.nc'
statistical_northness_file = 'processed_data/features/statistical_northness_features.nc'
statistical_eastness_file = 'processed_data/features/statistical_eastness_features.nc'

ds = xr.open_dataset(sinmod_file)
temp_ds = xr.open_dataset(temp_file)
salinity_ds = xr.open_dataset(salinity_file)
current_speed_ds = xr.open_dataset(current_speed_file)
statistical_northness_ds = xr.open_dataset(statistical_northness_file)
statistical_eastness_ds = xr.open_dataset(statistical_eastness_file)

SINMOD_features = xr.Dataset({
    'bottom_temperature_features': temp_ds["temperature_features"],
    'bottom_salinity_features': salinity_ds["salinity_features"],
    'bottom_current_features': current_speed_ds["current_speed_features"],
    'bottom_statistical_northness_features': statistical_northness["statistical_northness_features"],
    'bottom_statistical_eastness_features': statistical_eastness["statistical_eastness_features"],
})

temp_ds.close()
salinity_ds.close()
current_speed_ds.close()
statistical_northness_ds.close()
statistical_eastness_ds.close()
ds.close()

del temp_ds
del salinity_ds
del current_speed_ds
del statistical_northness_ds
del statistical_eastness_ds


SINMOD_features = SINMOD_features.reset_coords(drop=True)

print(SINMOD_features)

In [ ]:
#2: Get null land points as verification
null_land_points = ds['temperature'].isel(time=0, zc=0).isnull().sum().values

ocean_points = ds['temperature'].isel(time=0, zc=0).notnull().sum().values

print(f"Null land points: {null_land_points}")
print(f"Ocean points: {ocean_points}")
print(f"Total points: {null_land_points + ocean_points}")

In [ ]:
#3: Get the SINMOD crs and attach it to the dataset

def obtain_sinmod_crs(PhysStates_data):
    grid_mapping = PhysStates_data['grid_mapping']  #Replace 'grid_mapping' with the correct variable name if different
    grid_attrs = grid_mapping.attrs  

    #Print horizontal resolution if available
    horizontal_resolution = grid_attrs.get('horizontal_resolution', 'unknown')
    print(f"\nHorizontal resolution: {horizontal_resolution} meters")

    #Construct the CRS using the attributes
    crs_sinmod = CRS.from_proj4(
        f"+proj=stere "
        f"+lat_0={grid_attrs['latitude_of_projection_origin']} "
        f"+lat_ts={grid_attrs['standard_parallel']} "
        f"+lon_0={grid_attrs['straight_vertical_longitude_from_pole']} "
        f"+x_0={grid_attrs['false_easting']} "
        f"+y_0={grid_attrs['false_northing']} "
        f"+a={grid_attrs['semi_major_axis']} "
        f"+b={grid_attrs['semi_minor_axis']} "
        f"+units=m +no_defs"
    )

    print(f"\nSINMOD CRS: {crs_sinmod}")
    return crs_sinmod

midnor_crs = obtain_sinmod_crs(ds)

del(ds)

#Attach the crs to the SINMOD dataset
SINMOD_features = SINMOD_features.rio.write_crs(midnor_crs)

In [ ]:
#4: Align the SINMOD data with the bathymetry

tif_file = 'raw_data/EMOD-tifs/bathymetry_32N_Clip_sample.tif'

tif_files = ['raw_data/EMOD-tifs/aspect_cos.tif', 'raw_data/EMOD-tifs/aspect_sin.tif', 'raw_data/EMOD-tifs/bathymetry_32N_Clip_sample.tif', 'raw_data/EMOD-tifs/broad_BPI_std.tif',
             'raw_data/EMOD-tifs/fine_BPI_std.tif', 'raw_data/EMOD-tifs/log_ruggedness_1.tif', 'raw_data/EMOD-tifs/slope.tif']


def align_SINMOD_and_bathymetry(SINMOD_features, tif_file, resampling=Resampling.bilinear):
        
    with rioxarray.open_rasterio(tif_file) as tif:

        #Remove the band dimension from the tif data
        if 'band' in tif.dims:
            tif = tif.isel(band=0)

        SINMOD_features_reprojected = SINMOD_features.rio.reproject_match(tif, resampling=resampling)

        print(SINMOD_features_reprojected.rio.bounds())

         Make mask of NaN SINMOD values
        sinmod_mask = SINMOD_features_reprojected['bottom_temperature_features'][0].isnull()

        #Apply the mask to the EMOD data
        tif = tif.where(~sinmod_mask, np.nan)

        valid_mask = SINMOD_features_reprojected['bottom_temperature_features'][0].notnull()

        valid_columns = valid_mask.any(dim="y")  
        min_col = valid_columns.argmax().item()  #First non-NaN column from the left
        max_col = valid_columns.shape[0] - valid_columns[::-1].argmax().item() - 1  #First non-NaN column from the right

        #Find the first valid row (non-NaN) from the top (min row index)
        valid_rows = valid_mask.any(dim="x")  #Check for valid values in each row
        min_row = valid_rows.argmax().item()  #First non-NaN row from the top
        max_row = valid_rows.shape[0] - valid_rows[::-1].argmax().item() - 1  #Adjust for reverse indexing

        #Slice the raster to the bounding box of valid data
        clipped_SINMOD_features = SINMOD_features_reprojected.isel(x=slice(min_col, max_col + 1), y=slice(min_row, max_row + 1))
        
        clipped_tif = tif.rio.clip_box(minx=clipped_SINMOD_features.rio.bounds()[0]+1, 
                                        miny=clipped_SINMOD_features.rio.bounds()[1], 
                                        maxx=clipped_SINMOD_features.rio.bounds()[2], 
                                        maxy=clipped_SINMOD_features.rio.bounds()[3])
        
        clipped_tif = clipped_tif.reset_coords(drop=True)

    return clipped_SINMOD_features, clipped_tif

SINMOD_features_reprojected, tif = align_SINMOD_and_bathymetry(SINMOD_features, tif_file)

In [ ]:
#5: Handle missing values in the SINMOD dataset, make sure NaNs are consistent throughout

#Check that all features have the same null points
assert (SINMOD_features_reprojected['bottom_temperature_features'].isnull() == 
    SINMOD_features_reprojected['bottom_salinity_features'].isnull()).all()
assert (SINMOD_features_reprojected['bottom_temperature_features'].isnull() == 
    SINMOD_features_reprojected['bottom_current_features'].isnull()).all()

#Check that if one is not null, then they are all not null
assert (SINMOD_features_reprojected['bottom_temperature_features'].notnull() == 
    SINMOD_features_reprojected['bottom_salinity_features'].notnull()).all()
assert (SINMOD_features_reprojected['bottom_temperature_features'].notnull() == 
    SINMOD_features_reprojected['bottom_current_features'].notnull()).all()

#Ensure that tif is NaN everywhere SINMOD_features_reprojected is NaN
assert np.all(np.isnan(tif.values) == np.isnan(SINMOD_features_reprojected['bottom_temperature_features'][0].values))

#Ensure that tif is non-Nan everywhere SINMOD_features_reprojected is non-NaN
assert np.all(np.isfinite(tif.values) == np.isfinite(SINMOD_features_reprojected['bottom_temperature_features'][0].values))

In [ ]:
#6: Check tif properties
def check_tif_properties(tif_files):

    with rioxarray.open_rasterio(tif_files[0]) as tif:
        print("\n File to compare is: ", tif_files[0])
        ref_bounds, ref_crs, ref_res, ref_dims = tif.rio.bounds(), tif.rio.crs, tif.rio.resolution(), tif.shape

        #Check if all files match the reference properties
        for tif_file in tif_files[1:]:
            print("\n Checking file: ", tif_file)
            with rioxarray.open_rasterio(tif_file) as tif:
                if not (tif.rio.bounds() == ref_bounds and
                        tif.rio.crs == ref_crs and
                        tif.rio.resolution() == ref_res and
                        tif.shape == ref_dims):
                    print(f"Mismatch found in {tif_file}")
                    print(f"Expected bounds: {ref_bounds}, Found: {tif.rio.bounds()}")
                    print(f"Expected CRS: {ref_crs}, Found: {tif.rio.crs}")
                    print(f"Expected resolution: {ref_res}, Found: {tif.rio.resolution()}")
                    print(f"Expected dimensions: {ref_dims}, Found: {tif.shape}")
                    continue
                print("Properties match.")

check_tif_properties(tif_files)

In [ ]:
#7: Repeat for all the EMOD tifs and create a dataset for all the clipped tif files

EMOD_features = xr.Dataset()

for file in tif_files:
    SINMOD_temp, file_tif = align_SINMOD_and_bathymetry(SINMOD_features, file)
    
    #Add the clipped tif as a new variable in the dataset
    file_name = file.split("/")[-1].split(".")[0] + "_clipped"
    var_name = file.split("/")[-1].split(".")[0]
    EMOD_features[var_name] = file_tif

    file_tif.to_netcdf(f'processed_data/features/{file_name}.nc', mode='w')

    # Clear memory of file_tif and SINMOD_temp
    del file_tif
    del SINMOD_temp

EMOD_features = EMOD_features.reset_coords(drop=True)
EMOD_features.to_netcdf('processed_data/features/EMOD_features.nc', mode='w')

In [ ]:
#8: Create current_aspect_angle feature. Absolute difference between the current direction and the depth aspect direction.

aspect_bathymetry = EMOD_features['aspect_cos_clipped']
aspect_current = SINMOD_features_reprojected['bottom_statistical_northness_features'].sel(stat='mean')

SINMOD_features_reprojected['current_aspect_angle'] = abs(np.arccos(aspect_current) - np.arccos(aspect_bathymetry)) * 180 / np.pi

In [ ]:
#9: Standardizing the data

scaler = StandardScaler()

#Standardize the features
temperature_standardized = scaler.fit_transform(SINMOD_features_reprojected['temperature'].reshape(-1, 1)).reshape(SINMOD_features_reprojected['temperature'].shape)
salinity_standardized = scaler.fit_transform(SINMOD_features_reprojected['salinity'].reshape(-1, 1)).reshape(SINMOD_features_reprojected['salinity'].shape)
#repeat for all features

#Print the mean and standard deviation of the standardised features to verify
print(f"Standardised Temperature - Mean: {temperature_standardized.mean():.2f}, Std Dev: {temperature_standardized.std():.2f}")
print(f"Standardised Salinity - Mean: {salinity_standardized.mean():.2f}, Std Dev: {salinity_standardized.std():.2f}")
#repeat for all features

SINMOD_features_reprojected['temperature'] = temperature_standardized
SINMOD_features_reprojected['salinity'] = salinity_standardized
#repeat for all features

In [ ]:
#10: Saving the reprojected and standardized data
output_file = 'processed_data/features/ready-for-training/SINMOD_bottom_features.nc'

SINMOD_features_reprojected.to_netcdf(output_file, mode='w')